# Word2Vec

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('./train/train.csv')

### Clean Data

In [4]:
import re

# Lowercase and remove punctuation
to_remove = ['.', ',', '?', '!', ':', ';']
replacement = " "

pattern = "|".join(map(re.escape, to_remove))
df["text"] = df["text"].str.replace(pattern, replacement, regex=True)
df["text"] = df["text"].str.lower()

print(df.head())

        id                                               text author
0  id26305  this process  however  afforded me no means of...    EAP
1  id17569  it never once occurred to me that the fumbling...    HPL
2  id11008  in his left hand was a gold snuff box  from wh...    EAP
3  id27763  how lovely is spring as we looked from windsor...    MWS
4  id12958  finding nothing else  not even gold  the super...    HPL


### Word2Vec

In [5]:
import gensim
from gensim.models import Word2Vec

data = []

# Tokenize the text data
for i in range(len(df)):
    temp = []
    for j in range(len(df['text'][i].split())):
        temp.append(df['text'][i].split()[j])
    data.append(temp)

print(data[:2])

[['this', 'process', 'however', 'afforded', 'me', 'no', 'means', 'of', 'ascertaining', 'the', 'dimensions', 'of', 'my', 'dungeon', 'as', 'i', 'might', 'make', 'its', 'circuit', 'and', 'return', 'to', 'the', 'point', 'whence', 'i', 'set', 'out', 'without', 'being', 'aware', 'of', 'the', 'fact', 'so', 'perfectly', 'uniform', 'seemed', 'the', 'wall'], ['it', 'never', 'once', 'occurred', 'to', 'me', 'that', 'the', 'fumbling', 'might', 'be', 'a', 'mere', 'mistake']]


In [6]:
# Train Word2Vec model
model1 = gensim.models.Word2Vec(data, min_count=1,vector_size=100, window=5)

In [22]:
word1 = 'chicken'
word2 = 'butter'

word_sim = model1.wv.similarity(word1, word2) * 100
word_sim = word_sim

print(f"Similarity between {word1} and {word2}: {word_sim : .2f}%")

Similarity between chicken and butter:  85.88%
